In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from keras.metrics import top_k_categorical_accuracy

from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras.callbacks import TensorBoard
from keras.optimizers import Adam

from glob import glob
import json
from keras_helper import *

import gc
gc.enable()


Using TensorFlow backend.


In [2]:
def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    if (v1[0] == 0 and v1[1] == 0) or (v2[0] == 0 and v2[1] == 0):
        return 0
    
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def intervaled_cumsum(ar, new_flag):
    # Make a copy to be used as output array
    out = ar.copy()

    cumsum = 0
    for i in range(len(ar)):
        if new_flag[i] == 2:
            cumsum = 0
            out[i] = cumsum
        else:
            cumsum += ar[i]
            out[i] = cumsum

    return out

def _stack_it(raw_strokes):
    """preprocess the string and make 
    a standard Nx3 stroke vector"""
    stroke_vec = json.loads(raw_strokes) # string->list

    # unwrap the list
    in_strokes = [(xi,yi,i) for i,(x,y) in enumerate(stroke_vec) for xi,yi in zip(x,y)]
    c_strokes = np.stack(in_strokes)
    
    # replace stroke id with 1 for continue, 2 for new
    c_strokes[:,2] += 1 # since 0 is no stroke
    new_flag = np.array([1]+np.diff(c_strokes[:,2]).tolist()) + 1

    # calc distance 
    x_diff = [0] + np.diff(c_strokes[:,0]).tolist()
    y_diff = [0] + np.diff(c_strokes[:,1]).tolist()
    distance = np.sqrt(np.power(x_diff, 2) + np.power(y_diff, 2)).astype(np.uint32)
    
    # calc length for one stroke
    length = np.bincount(c_strokes[:,2], weights=distance).astype(np.uint32)
    leng = np.zeros_like(distance)
    for i in range(1, len(length)):
        leng[c_strokes[:,2] == i] = length[i]
    
    c_strokes = np.column_stack((c_strokes, new_flag, distance, leng))
    c_strokes[c_strokes[:,3] == 2,4] = 0
    
    len_cumsum = intervaled_cumsum(c_strokes[:,4],c_strokes[:,2])  
    c_strokes = np.column_stack((c_strokes, len_cumsum))

    # pad the strokes with zeros
    return pad_sequences(c_strokes.swapaxes(0, 1), 
                         maxlen=STROKE_COUNT, 
                         padding='post').swapaxes(0, 1)
 
def _stack_it9(raw_strokes):
    stroke_vec = json.loads(raw_strokes)
    for d in stroke_vec:
        angle = []
        for i in range(len(d[0])):
            if i < 2:
                angle.append(0)
            else:
                v1 = (d[0][i-1] - d[0][i-2],d[1][i-1] - d[1][i-2])
                v2 = (d[0][i] - d[0][i-1],d[1][i] - d[1][i-1])
                a = angle_between(v1,v2)
                a = int(a)
                angle.append(a)

        d.append(angle)

    # unwrap the list
    in_strokes = [(xi,yi,ai,i) for i,(x,y,a) in enumerate(stroke_vec) for xi,yi,ai in zip(x,y,a)]
    c_strokes = np.stack(in_strokes)

    # replace stroke id with 1 for continue, 2 for new
    c_strokes[:,3] += 1 # since 0 is no stroke
    new_flag = np.array([1]+np.diff(c_strokes[:,3]).tolist()) + 1

    # calc distance 
    x_diff = [0] + np.diff(c_strokes[:,0]).tolist()
    y_diff = [0] + np.diff(c_strokes[:,1]).tolist()
    distance = np.sqrt(np.power(x_diff, 2) + np.power(y_diff, 2)).astype(np.uint32)

    # calc length for one stroke
    length = np.bincount(c_strokes[:,3], weights=distance).astype(np.uint32)
    leng = np.zeros_like(distance)
    for i in range(1, len(length)):
        leng[c_strokes[:,3] == i] = length[i]

    c_strokes = np.column_stack((c_strokes, new_flag, distance, leng))
    c_strokes[c_strokes[:,4] == 2,5] = 0

    angle_cumsum = intervaled_cumsum(c_strokes[:,2],c_strokes[:,4])
    length_cumsum = intervaled_cumsum(c_strokes[:,5],c_strokes[:,4])
    c_strokes = np.column_stack((c_strokes, angle_cumsum, length_cumsum))

    # pad the strokes with zeros
    return pad_sequences(c_strokes.swapaxes(0, 1), 
                         maxlen=STROKE_COUNT, 
                         padding='post').swapaxes(0, 1)

In [3]:
# val_df = pd.read_csv('../input/valid.csv', nrows=10)
# _stack_it(val_df.iloc[1]['drawing'])[:30]

In [4]:
def image_generator_xd(batchsize, df_path = '../input/train_all.csv'):
    while True:
        for df in pd.read_csv(df_path, chunksize=batchsize):
            df['drawing'] = df['drawing'].map(_stack_it9)
            x2 = np.stack(df['drawing'], 0)
            y = keras.utils.to_categorical(df['word'], num_classes=NCATS)
            yield x2, y

def df_to_image_array_xd(df):
    df['drawing'] = df['drawing'].map(_stack_it9)
    x2 = np.stack(df['drawing'], 0)
    return x2

In [5]:
STROKE_COUNT = 100
NCATS = 340
TOT = 46613580 - 340000
EPOCHS = 70
batchsize = 512
STEPS = TOT / EPOCHS / batchsize 

conv_filters = 64
lstm_units = 384
lstm_cnt = 3
dropout = 0.2

model_prefix = 'lstm_conv{}_units{}_layer{}_dropout{}_feats9_smallconv'.format(conv_filters, lstm_units, lstm_cnt, dropout)
print(model_prefix)
check_path = 'models/best_{}.hdf5'.format(model_prefix)

lstm_conv64_units384_layer3_dropout0.2_feats9_smallconv


In [6]:
train_datagen = image_generator_xd(batchsize=batchsize, df_path = '../input/train_all.csv')
val_df = pd.read_csv('../input/valid.csv')
x_val = df_to_image_array_xd(val_df)
y_val = keras.utils.to_categorical(val_df.word, num_classes=NCATS)

In [7]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv1D, Dense, Dropout, Bidirectional
from keras.layers import CuDNNLSTM
#if len(get_available_gpus())>0:
    # https://twitter.com/fchollet/status/918170264608817152?lang=en
#    from keras.layers import CuDNNLSTM as LSTM # this one is about 3x faster on GPU instances
stroke_read_model = Sequential()
stroke_read_model.add(BatchNormalization(input_shape = (None,)+(9,)))
# filter count and length are taken from the script https://github.com/tensorflow/models/blob/master/tutorials/rnn/quickdraw/train_model.py
stroke_read_model.add(Conv1D(conv_filters, (3,), activation = 'relu'))
stroke_read_model.add(Conv1D(conv_filters*2, (3,), activation = 'relu'))
stroke_read_model.add(Conv1D(conv_filters*3, (3,), activation = 'relu'))
stroke_read_model.add(Conv1D(conv_filters*4, (3,), activation = 'relu'))

for i in range(lstm_cnt - 1):
    print('return_sequences', i != (lstm_cnt - 1))
    stroke_read_model.add(Bidirectional(CuDNNLSTM(lstm_units, return_sequences = True)))
stroke_read_model.add(Bidirectional(CuDNNLSTM(lstm_units, return_sequences = False)))
                                    
stroke_read_model.add(Dense(512, activation = 'relu'))
stroke_read_model.add(Dropout(dropout))
stroke_read_model.add(Dense(NCATS, activation = 'softmax'))
stroke_read_model.compile(optimizer = Adam(lr=0.0002), 
                          loss = 'categorical_crossentropy', 
                          metrics = ['categorical_accuracy', top_3_accuracy])
stroke_read_model.summary()

return_sequences True
return_sequences True
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, None, 9)           36        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 64)          1792      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         24704     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 192)         73920     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 256)         147712    
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 768)         1972224   
_________________________________________________________________
bidirectional_2 (Bidirection (No

In [8]:
# stroke_read_model.load_weights('models/best_lstm_conv64_units384_layer3_dropout0.2_feats9_noconvdrop2.hdf5')
# stroke_read_model.load_weights('models/best_lstm_conv64_units384_layer3_dropout0.2_feats9_noconvdrop.hdf5')

# stroke_read_model.evaluate(x_val, y_val, batch_size=512)

In [9]:
checkpoint = ModelCheckpoint(check_path, monitor='val_categorical_accuracy', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, 
                                   verbose=1, mode='max', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_categorical_accuracy", 
                      mode="max", 
                      patience=20) 
board =TensorBoard(log_dir='./log/{}'.format(model_prefix))
callbacks_list = [checkpoint, early, reduceLROnPlat, board]

/home/kownse/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [10]:
# snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS, 
#                                        nb_snapshots=1,
#                                        size = 384,
#                                        init_lr=0.0002, 
#                                        fold=0)
 
# callbacks_list = snapshot.get_callbacks(model_prefix = model_prefix)

In [11]:
hist = stroke_read_model.fit_generator(train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
                        validation_data=(x_val, y_val),
                      callbacks = callbacks_list)

Epoch 1/70
1292/1291 [==============================] - 974s 754ms/step - loss: 3.7968 - categorical_accuracy: 0.1988 - top_3_accuracy: 0.3501 - val_loss: 2.8564 - val_categorical_accuracy: 0.3433 - val_top_3_accuracy: 0.5483

Epoch 00001: val_categorical_accuracy improved from -inf to 0.34329, saving model to models/best_lstm_conv64_units384_layer3_dropout0.2_feats9_smallconv.hdf5
Epoch 2/70
1292/1291 [==============================] - 968s 749ms/step - loss: 2.5789 - categorical_accuracy: 0.3991 - top_3_accuracy: 0.6057 - val_loss: 2.1704 - val_categorical_accuracy: 0.4833 - val_top_3_accuracy: 0.6896

Epoch 00002: val_categorical_accuracy improved from 0.34329 to 0.48332, saving model to models/best_lstm_conv64_units384_layer3_dropout0.2_feats9_smallconv.hdf5
Epoch 3/70
1292/1291 [==============================] - 968s 749ms/step - loss: 2.0544 - categorical_accuracy: 0.5072 - top_3_accuracy: 0.7116 - val_loss: 1.7922 - val_categorical_accuracy: 0.5642 - val_top_3_accuracy: 0.7630



KeyboardInterrupt: 

In [ ]:
test = pd.read_csv('../input/test_simplified.csv')
x_test = df_to_image_array_xd(test)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}

In [ ]:
 def doodle_predict(model, x_test):
    model.load_weights(model_path)

    test_predictions = model.predict(x_test, batch_size=512, verbose=1)
    top3 = preds2catids(test_predictions)
    top3cats = top3.replace(id2cat)
    test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
    submission = test[['key_id', 'word']]

    import kaggle_util
    kaggle_util.save_result(submission,  
                            '../result/{}.csv'.format(model_prefix), 
                            'quickdraw-doodle-recognition', 
                            send=True, index=False) 

In [ ]:
doodle_predict(stroke_read_model, check_path, x_test)

In [ ]:
test_predictions = stroke_read_model.predict(x_test, batch_size=512, verbose=1)
np.save('../result/{}.npy'.format(model_prefix), test_predictions)